In [267]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn import preprocessing
from sklearn import linear_model
import statsmodels.formula.api as smf
import re
%matplotlib inline

In [180]:
law_enf = pd.read_excel("okle_new_york.xls", skiprows=4,
                       names=["City", "Population", "ViolentCrime",
                             "Murder", "RapeR", "RapeL", "Robbery",
                             "AggravatedAssault", "PropertyCrime",
                             "Burglary", "LarcenyTheft", "MotorTheft",
                             "Arson"])
law_enf_2014 = pd.read_excel("okle_new_york_2014.xls", skiprows=4,
                            names=["City", "Population", "ViolentCrime",
                             "Murder", "RapeR", "RapeL", "Robbery",
                             "AggravatedAssault", "PropertyCrime",
                             "Burglary", "LarcenyTheft", "MotorTheft",
                             "Arson"])

law_enf = law_enf.rename(columns=lambda x:x.strip())
law_enf = law_enf.rename(columns=lambda x: x.replace('\n', ''))
law_enf = law_enf.rename(columns=lambda x: x.replace(' ', ''))

law_enf_2014 = law_enf_2014.rename(columns=lambda x:x.strip())
law_enf_2014 = law_enf_2014.rename(columns=lambda x: x.replace('\n', ''))
law_enf_2014 = law_enf_2014.rename(columns=lambda x: x.replace(' ', ''))

In [181]:
law_enf.head()

,City,Population,ViolentCrime,Murder,RapeR,RapeL,Robbery,AggravatedAssault,PropertyCrime,Burglary,LarcenyTheft,MotorTheft,Arson
0,Adams Village,1861.0,0.0,0.0,NaN,0.0,0.0,0.0,12.0,2.0,10.0,0.0,0.0
1,Addison Town and Village,2577.0,3.0,0.0,NaN,0.0,0.0,3.0,24.0,3.0,20.0,1.0,0.0
2,Akron Village,2846.0,3.0,0.0,NaN,0.0,0.0,3.0,16.0,1.0,15.0,0.0,0.0
3,Albany,97956.0,791.0,8.0,NaN,30.0,227.0,526.0,4090.0,705.0,3243.0,142.0,NaN
4,Albion Village,6388.0,23.0,0.0,NaN,3.0,4.0,16.0,223.0,53.0,165.0,5.0,NaN


In [182]:
law_enf_2014.head()

,City,Population,ViolentCrime,Murder,RapeR,RapeL,Robbery,AggravatedAssault,PropertyCrime,Burglary,LarcenyTheft,MotorTheft,Arson
0,Adams Village,1851.0,0.0,0.0,NaN,0.0,0.0,0.0,11.0,1.0,10.0,0.0,0.0
1,Addison Town and Village,2568.0,2.0,0.0,NaN,0.0,1.0,1.0,49.0,1.0,47.0,1.0,0.0
2,Afton Village4,820.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,Akron Village,2842.0,1.0,0.0,NaN,0.0,0.0,1.0,17.0,0.0,17.0,0.0,0.0
4,Albany4,98595.0,802.0,8.0,54.0,NaN,237.0,503.0,3888.0,683.0,3083.0,122.0,12.0


In [183]:
law_enf['PopulationSquared'] = law_enf['Population'] ** 2
law_enf['Murder'] = law_enf['Murder'] > 0
law_enf['Robbery'] = law_enf['Robbery'] > 0

law_enf_2014['PopulationSquared'] = law_enf_2014['Population'] ** 2
law_enf_2014['Murder'] = law_enf_2014['Murder'] > 0
law_enf_2014['Robbery'] = law_enf_2014['Robbery'] > 0

In [184]:
law_enf = law_enf.iloc[:348]
law_enf_2014 = law_enf_2014.iloc[:369]

In [185]:
law_enf = law_enf.drop(['RapeR', 'Arson'], axis=1)
law_enf_2014 = law_enf_2014.drop(['RapeR', 'Arson'], axis=1)

In [186]:
print(law_enf[law_enf['PropertyCrime'] > 100000]["City"])
print(law_enf_2014[law_enf_2014['PropertyCrime'] > 100000]["City"])

law_enf_wo_ny = law_enf.drop(216)
law_enf_2014_wo_ny = law_enf_2014.drop(227)

216    New York
Name: City, dtype: object
227    New York4
Name: City, dtype: object


In [187]:
linear_formula = "PropertyCrime ~ Population"
lm = smf.ols(formula=linear_formula, data=law_enf_wo_ny).fit()
print(lm.params)
print(lm.pvalues)
print(lm.rsquared)

#RSS sum of squared residuals = SSE_F 
#TSS total sum of squares = SSE_R 
#       which is RSS of "reduced model" = no features aka mean

lm.predict(law_enf_wo_ny)
lm.summary()

Intercept    -155.020489
Population      0.033890
dtype: float64
Intercept      3.340745e-07
Population    5.644411e-118
dtype: float64
0.7872222002523548


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          PropertyCrime   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.787
Method:                 Least Squares   F-statistic:                     1276.
Date:                Wed, 16 May 2018   Prob (F-statistic):          5.64e-118
Time:                        22:25:03   Log-Likelihood:                -2632.1
No. Observations:                 347   AIC:                             5268.
Df Residuals:                     345   BIC:                             5276.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -155.0205     29.784     -5.205      0.000    -213.601     -96.440
Population     0.0339      0.001     35.727      0.000       0.032       0.036
==============================================================================
Omnibus:                      127.397   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18319.959
Skew:                          -0.124   Prob(JB):                         0.00
Kurtosis:                      38.595   Cond. No.                     3.65e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.65e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [188]:
def compute_RSS(model, data):
    predictions = model.predict(data)
    squared_residuals = (data['PropertyCrime'] - predictions) ** 2
    return squared_residuals.sum()

In [189]:
def compute_TSS(data):
    return ((data['PropertyCrime'] - data['PropertyCrime'].mean()) ** 2).sum()

In [190]:
def F_statistic(p, data, model):
    n = len(data)
    RSS = compute_RSS(model, data)
    TSS = compute_TSS(data)
    return ((TSS - RSS) / p) /\
            (RSS / (n - p - 1))

In [1]:
def list_to_form(list):
    return '+'.join(list)

In [273]:
def mixed_selection(data, validation_data):
    tuned_list = []
    features = list(data.drop(['PropertyCrime', 'City'], axis=1).columns)
    best_lm = None
    test_formula = None
    for i in range(len(features)):
        best_feature = None
        best_RSS = float("inf")
        init_formula = "PropertyCrime ~ " + list_to_form(tuned_list)
        for feature in features:
            test_formula = init_formula
            if len(tuned_list) > 0:
                test_formula = test_formula + "+" + feature
            else:
                test_formula = test_formula + feature
            test_lm = smf.ols(formula=test_formula, data=law_enf_wo_ny).fit()
            test_RSS = compute_RSS(lm, validation_data)
            if test_RSS < best_RSS:
                best_RSS = test_RSS
                best_feature = feature
        print("add", feature)
        tuned_list.append(feature)
        features.remove(feature)
        best_formula = "PropertyCrime ~ " + list_to_form(tuned_list)
        best_lm = smf.ols(formula=best_formula, data=law_enf_wo_ny).fit()
        pvalues = best_lm.pvalues.rename(lambda x: re.sub(r"\[.*\]", "", x))
        for feature in tuned_list:
            if feature in pvalues and pvalues[feature] > 0.05:
                print("remove", feature)
                tuned_list.remove(feature)
                best_formula = "PropertyCrime ~ " + list_to_form(tuned_list)
                best_lm = smf.ols(formula=best_formula, data=law_enf_wo_ny).fit()
    return best_lm, tuned_list

In [274]:
lm, tuned_list = mixed_selection(law_enf_wo_ny, law_enf_2014_wo_ny)
print(tuned_list)
lm.summary()

add PopulationSquared
add MotorTheft
remove PopulationSquared
add LarcenyTheft
add Burglary
add AggravatedAssault
remove AggravatedAssault
add Robbery
remove Robbery
add RapeL
remove RapeL
add Murder
add ViolentCrime
remove ViolentCrime
add Population
remove Murder
['MotorTheft', 'LarcenyTheft', 'Burglary', 'Population']


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          PropertyCrime   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.765e+30
Date:                Wed, 16 May 2018   Prob (F-statistic):               0.00
Time:                        23:54:50   Log-Likelihood:                 8411.1
No. Observations:                 347   AIC:                        -1.681e+04
Df Residuals:                     342   BIC:                        -1.679e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept    -7.311e-12   4.98e-13    -14.687      0.000   -8.29e-12   -6.33e-12
MotorTheft       1.0000   3.04e-14   3.29e+13      0.000       1.000       1.000
LarcenyTheft     1.0000   2.65e-15   3.77e+14      0.000       1.000       1.000
Burglary         1.0000   1.03e-14   9.71e+13      0.000       1.000       1.000
Population     4.51e-17   3.88e-17      1.162      0.246   -3.13e-17    1.21e-16
==============================================================================
Omnibus:                      297.606   Durbin-Watson:                   0.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            17416.375
Skew:                           3.088   Prob(JB):                         0.00
Kurtosis:                      37.153   Cond. No.                     4.02e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.02e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
# TODO implement cross validation and hold out set testing